In [1]:
import requests
from lxml import etree
import pandas as pd
import numpy as np
import re
import time
from bs4 import BeautifulSoup

In [21]:
base_url = "https://sofifa.com/players?offset="
headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
columns = ['ID', 'picture','Nationality' ,'name', 'age','position', 'overall', 'potential', 'club', 'value','wage']
data = pd.DataFrame(columns = columns)

In [22]:
url = 'https://sofifa.com/players?offset=0'
resp = requests.get(url=url, headers=headers)
html = etree.HTML(resp.text)
tr1 = html.xpath('/html/body/div[1]/div/div/div[1]/div/table/tbody/tr')[2]
nationality=tr1.xpath('./td[2]/a[1]/div/img/@title')[0].strip()
nationality

'France'

In [25]:
count=0
for offset in range(0, 300):
    url = base_url + str(offset * 61)
    resp = requests.get(url=url, headers=headers)
    html = etree.HTML(resp.text)
    trs = html.xpath('/html/body/div[1]/div/div/div[1]/div/table/tbody/tr')
    for tr1 in trs:
        URL = tr1.xpath('./td[2]/a[1]/@href')[0].strip()
        ID=URL.split('/')[2]
        nationality=tr1.xpath('./td[2]/a[1]/div/img/@title')[0].strip()
        if len(tr1.xpath('./td[1]/figure/img/@data-src')):
            picture = tr1.xpath('./td[1]/figure/img/@data-src')[0].strip()
        else:
            picture = np.nan
        name = tr1.xpath('./td[2]/a[1]/@data-tooltip')[0].strip()
        age = tr1.xpath('./td[3]/text()')[0].strip()
        position = tr1.xpath('./td[2]/a[2]/span/text()')[0].strip()
        overall = tr1.xpath('./td[4]/span/text()')[0].strip()
        potential = tr1.xpath('./td[5]/span/text()')[0].strip()
        if len(tr1.xpath('./td[6]/div/a/text()')):
            club = tr1.xpath('./td[6]/div/a/text()')[0].strip()
        else:
            club = np.nan
        value = tr1.xpath('./td[7]/text()')[0].strip()
        wage = tr1.xpath('./td[8]/text()')[0].strip()
        
        player_data = pd.DataFrame(
            [[ID,picture,nationality,name,age,position,overall,potential,club,value,wage]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
 

In [26]:
data = data.drop_duplicates()
data.shape

(15785, 11)

In [27]:
detailed_data=[]
count=0
skill_map = {}
for ID in data['ID']:   
    
    url = 'https://sofifa.com/player/'+str(ID)
    print(url)
    resp = requests.get(url=url, headers=headers)
    html = etree.HTML(resp.text)
    skill_map = {}
    if len(html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[8]/div/ul/li[6]/span/text()')):

        link_1 = html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[2]/div/div[1]/div/ul')[0]
        skill_map['weak_foot'] = link_1.xpath('./li[2]/text()')[0].strip()
        skill_map['skill_moves'] = link_1.xpath('./li[3]/text()')[0].strip()
        skill_map['reputation'] = link_1.xpath('./li[4]/text()')[0].strip()
        meta_data= html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[1]/div/div/div/text()')
        a=str(meta_data)
        skill_map['weight']=re.findall(r"\d+",a)[-1]
        skill_map['height']=re.findall(r"\d+",a)[-3]+'.'+re.findall(r"\d+",a)[-2]


        for i in range(5,8):
            number=str(i)
            key = link_1.xpath('./li[' + number + ']/label/text()')[0].strip()
            value = link_1.xpath('./li[' + number + ']/span/text()')[0].strip()
            skill_map[key]=value
        # some player do not have release clause
        if len(link_1.xpath('./li[8]/span/text()')):
            skill_map['release_clause'] = link_1.xpath('./li[8]/span/text()')[0].strip()
            skill_map['ID'] = link_1.xpath('./li[9]/text()')[0].strip()
        else:
            skill_map['release_clause'] = np.nan
            skill_map['ID'] = link_1.xpath('./li[8]/text()')[0].strip()

        for div in range(4,8):
            number = str(div)
            skill_datas = html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[' + number + ']/div/ul/li')
            for skills in skill_datas:
                #检查存在加号与否
                if len(re.findall(r"\d",skills.xpath('./span[2]/text()')[0].strip())):
                    value = skills.xpath('./span/text()')[0].strip()
                    key = skills.xpath('./span[3]/text()')[0].strip()
                    skill_map[key]=value
                else:
                    value = skills.xpath('./span/text()')[0].strip()
                    key = skills.xpath('./span[2]/text()')[0].strip()
                    skill_map[key]=value


        skill_datas = html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[8]/div/ul')[0]
        for i in range(1,6):
                #检查存在加号与否
                number=str(i)
                if len(re.findall(r"\d",skill_datas.xpath('./li[' + number + ']/span[2]/text()')[0].strip())):
                    value = skill_datas.xpath('./li[' + number + ']/span[1]/text()')[0].strip()
                    key = skill_datas.xpath('./li[' + number + ']/span[3]/text()')[0].strip()
                    skill_map[key]=value
                else:
                    value = skill_datas.xpath('./li[' + number + ']/span[1]/text()')[0].strip()
                    key = skill_datas.xpath('./li[' + number + ']/span[2]/text()')[0].strip()
                    skill_map[key]=value
        #some pages do not have composure
        if len(html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[8]/div/ul/li[6]/span/text()')):
            skill_map['composure'] = html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[8]/div/ul/li[6]/span/text()')[0].strip()
        else:
            skill_map['composure']=np.nan

        for div in range(9,10):
            number = str(div)
            skill_datas = html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[' + number + ']/div/ul/li')
            for skills in skill_datas:
                #检查是否有加号
                if len(re.findall(r"\d",skills.xpath('./span[2]/text()')[0].strip())):
                    value = skills.xpath('./span/text()')[0].strip()
                    key = skills.xpath('./span[3]/text()')[0].strip()
                    skill_map[key]=value
                else:
                    value = skills.xpath('./span/text()')[0].strip()
                    key = skills.xpath('./span[2]/text()')[0].strip()
                    skill_map[key]=value


        skill_datas = html.xpath('/html/body/div[2]/div/div/div[1]/div[1]/div[10]/div/ul')[0]
        for i in range(1,6):
            number=str(i)

            value = skill_datas.xpath('./li[' + number + ']/span/text()')[0].strip()
            key = skill_datas.xpath('./li[' + number + ']/text()')[0].strip()
            skill_map[key]=value
        detailed_data.append(skill_map)
        time.sleep(0.5)
        count+=1
        print(count)
    else:
        pass

https://sofifa.com/player/177620
https://sofifa.com/player/255043
1
https://sofifa.com/player/228093
2
https://sofifa.com/player/247361
3
https://sofifa.com/player/230872
4
https://sofifa.com/player/252145
5
https://sofifa.com/player/235212
6
https://sofifa.com/player/237000
7
https://sofifa.com/player/45661
https://sofifa.com/player/245940
8
https://sofifa.com/player/222227
9
https://sofifa.com/player/233731
10
https://sofifa.com/player/20801
11
https://sofifa.com/player/231936
12
https://sofifa.com/player/239085
13
https://sofifa.com/player/253177
14
https://sofifa.com/player/245541
15
https://sofifa.com/player/228251
16
https://sofifa.com/player/204539
17
https://sofifa.com/player/206517
18
https://sofifa.com/player/232488
19
https://sofifa.com/player/232938
20
https://sofifa.com/player/238744
21
https://sofifa.com/player/237692
22
https://sofifa.com/player/158023
23
https://sofifa.com/player/251804
24
https://sofifa.com/player/237995
25
https://sofifa.com/player/204923
26
https://s

223
https://sofifa.com/player/234060
224
https://sofifa.com/player/241928
225
https://sofifa.com/player/216267
226
https://sofifa.com/player/229880
227
https://sofifa.com/player/41236
228
https://sofifa.com/player/229906
229
https://sofifa.com/player/242249
230
https://sofifa.com/player/251810
231
https://sofifa.com/player/253473
232
https://sofifa.com/player/178509
233
https://sofifa.com/player/188567
234
https://sofifa.com/player/226271
235
https://sofifa.com/player/230142
236
https://sofifa.com/player/241095
237
https://sofifa.com/player/242434
238
https://sofifa.com/player/173731
239
https://sofifa.com/player/246791
240
https://sofifa.com/player/205569
241
https://sofifa.com/player/207865
242
https://sofifa.com/player/236610
243
https://sofifa.com/player/209331
244
https://sofifa.com/player/211302
245
https://sofifa.com/player/233795
246
https://sofifa.com/player/240679
247
https://sofifa.com/player/241487
248
https://sofifa.com/player/181375
249
https://sofifa.com/player/201535
25

445
https://sofifa.com/player/179944
446
https://sofifa.com/player/208830
447
https://sofifa.com/player/225953
448
https://sofifa.com/player/241508
449
https://sofifa.com/player/243392
450
https://sofifa.com/player/243854
451
https://sofifa.com/player/204970
452
https://sofifa.com/player/233866
453
https://sofifa.com/player/239380
454
https://sofifa.com/player/239837
455
https://sofifa.com/player/247106
456
https://sofifa.com/player/41
457
https://sofifa.com/player/248386
458
https://sofifa.com/player/204082
459
https://sofifa.com/player/207877
460
https://sofifa.com/player/233096
461
https://sofifa.com/player/234612
462
https://sofifa.com/player/246785
463
https://sofifa.com/player/253283
464
https://sofifa.com/player/201399
465
https://sofifa.com/player/202884
466
https://sofifa.com/player/203263
467
https://sofifa.com/player/206585
468
https://sofifa.com/player/211117
469
https://sofifa.com/player/228815
470
https://sofifa.com/player/231866
471
https://sofifa.com/player/237321
472
h

667
https://sofifa.com/player/242429
668
https://sofifa.com/player/173432
669
https://sofifa.com/player/178005
670
https://sofifa.com/player/184087
671
https://sofifa.com/player/205929
672
https://sofifa.com/player/206467
673
https://sofifa.com/player/210406
674
https://sofifa.com/player/211515
675
https://sofifa.com/player/212419
676
https://sofifa.com/player/213666
677
https://sofifa.com/player/216460
678
https://sofifa.com/player/225964
679
https://sofifa.com/player/228708
680
https://sofifa.com/player/232440
681
https://sofifa.com/player/235008
682
https://sofifa.com/player/236441
683
https://sofifa.com/player/242187
684
https://sofifa.com/player/244915
685
https://sofifa.com/player/247028
686
https://sofifa.com/player/257270
687
https://sofifa.com/player/258781
688
https://sofifa.com/player/201519
689
https://sofifa.com/player/204233
690
https://sofifa.com/player/209297
691
https://sofifa.com/player/221697
692
https://sofifa.com/player/223885
693
https://sofifa.com/player/225085
6

887
https://sofifa.com/player/245426
888
https://sofifa.com/player/246053
889
https://sofifa.com/player/251401
890
https://sofifa.com/player/254470
891
https://sofifa.com/player/255862
892
https://sofifa.com/player/260247
893
https://sofifa.com/player/146536
894
https://sofifa.com/player/177413
895
https://sofifa.com/player/182493
896
https://sofifa.com/player/190761
897
https://sofifa.com/player/190815
898
https://sofifa.com/player/191043
899
https://sofifa.com/player/193290
900
https://sofifa.com/player/202371
901
https://sofifa.com/player/204935
902
https://sofifa.com/player/150724
903
https://sofifa.com/player/162895
904
https://sofifa.com/player/171897
905
https://sofifa.com/player/192883
906
https://sofifa.com/player/198329
907
https://sofifa.com/player/200260
908
https://sofifa.com/player/208450
909
https://sofifa.com/player/211784
910
https://sofifa.com/player/213629
911
https://sofifa.com/player/225508
912
https://sofifa.com/player/230829
913
https://sofifa.com/player/235944
9

...

14225
https://sofifa.com/player/248114
14226
https://sofifa.com/player/248136
14227
https://sofifa.com/player/248183
14228
https://sofifa.com/player/248186
14229
https://sofifa.com/player/248213
14230
https://sofifa.com/player/248236
14231
https://sofifa.com/player/248256
14232
https://sofifa.com/player/248283
14233
https://sofifa.com/player/248345
14234
https://sofifa.com/player/248347
14235
https://sofifa.com/player/246902
14236
https://sofifa.com/player/246919
14237
https://sofifa.com/player/246926
14238
https://sofifa.com/player/246940
14239
https://sofifa.com/player/246951
14240
https://sofifa.com/player/246974
14241
https://sofifa.com/player/246987
14242
https://sofifa.com/player/246990
14243
https://sofifa.com/player/246996
14244
https://sofifa.com/player/246999
14245
https://sofifa.com/player/247017
14246
https://sofifa.com/player/247033
14247
https://sofifa.com/player/247067
14248
https://sofifa.com/player/247072
14249
https://sofifa.com/player/247076
14250
https://sofifa.com/

14436
https://sofifa.com/player/244905
14437
https://sofifa.com/player/244912
14438
https://sofifa.com/player/244927
14439
https://sofifa.com/player/244929
14440
https://sofifa.com/player/244933
14441
https://sofifa.com/player/244958
14442
https://sofifa.com/player/244976
14443
https://sofifa.com/player/244998
14444
https://sofifa.com/player/245034
14445
https://sofifa.com/player/245040
14446
https://sofifa.com/player/245045
14447
https://sofifa.com/player/245056
14448
https://sofifa.com/player/245071
14449
https://sofifa.com/player/245088
14450
https://sofifa.com/player/245092
14451
https://sofifa.com/player/245129
14452
https://sofifa.com/player/245151
14453
https://sofifa.com/player/245166
14454
https://sofifa.com/player/245170
14455
https://sofifa.com/player/245219
14456
https://sofifa.com/player/245224
14457
https://sofifa.com/player/245249
14458
https://sofifa.com/player/245257
14459
https://sofifa.com/player/245261
14460
https://sofifa.com/player/245263
14461
https://sofifa.com/

14647
https://sofifa.com/player/243278
14648
https://sofifa.com/player/243280
14649
https://sofifa.com/player/243284
14650
https://sofifa.com/player/243305
14651
https://sofifa.com/player/243306
14652
https://sofifa.com/player/243312
14653
https://sofifa.com/player/243314
14654
https://sofifa.com/player/243320
14655
https://sofifa.com/player/242057
14656
https://sofifa.com/player/242062
14657
https://sofifa.com/player/242068
14658
https://sofifa.com/player/242071
14659
https://sofifa.com/player/242077
14660
https://sofifa.com/player/242082
14661
https://sofifa.com/player/242100
14662
https://sofifa.com/player/242102
14663
https://sofifa.com/player/242108
14664
https://sofifa.com/player/242117
14665
https://sofifa.com/player/242149
14666
https://sofifa.com/player/242155
14667
https://sofifa.com/player/242156
14668
https://sofifa.com/player/242183
14669
https://sofifa.com/player/242197
14670
https://sofifa.com/player/242204
14671
https://sofifa.com/player/242209
14672
https://sofifa.com/

14858
https://sofifa.com/player/240355
14859
https://sofifa.com/player/240360
14860
https://sofifa.com/player/240385
14861
https://sofifa.com/player/240388
14862
https://sofifa.com/player/240409
14863
https://sofifa.com/player/240410
14864
https://sofifa.com/player/240420
14865
https://sofifa.com/player/240428
14866
https://sofifa.com/player/240437
14867
https://sofifa.com/player/240457
14868
https://sofifa.com/player/240473
14869
https://sofifa.com/player/240481
14870
https://sofifa.com/player/240492
14871
https://sofifa.com/player/240493
14872
https://sofifa.com/player/240511
14873
https://sofifa.com/player/240512
14874
https://sofifa.com/player/240526
14875
https://sofifa.com/player/240527
14876
https://sofifa.com/player/240529
14877
https://sofifa.com/player/240531
14878
https://sofifa.com/player/240543
14879
https://sofifa.com/player/240545
14880
https://sofifa.com/player/240547
14881
https://sofifa.com/player/240562
14882
https://sofifa.com/player/240563
14883
https://sofifa.com/

15069
https://sofifa.com/player/238862
15070
https://sofifa.com/player/238868
15071
https://sofifa.com/player/238882
15072
https://sofifa.com/player/238894
15073
https://sofifa.com/player/238919
15074
https://sofifa.com/player/238930
15075


In [28]:
detailed_data=pd.DataFrame(detailed_data)

In [29]:
detailed_data=detailed_data.iloc[:,0:44]

In [30]:
detailed_data.head(10)

,weak_foot,skill_moves,reputation,weight,height,Work Rate,Body Type,Real Face,release_clause,ID,...,Penalties,composure,Defensive Awareness,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,4,2,1,220,6.2,High/ Low,Stocky (185+),No,NaN,255043,...,65,52,15,15,10,11,9,15,11,13
1,3,3,1,192,6.4,Medium/ Medium,Normal (185+),No,€53.2M,228093,...,70,74,43,49,37,11,13,14,9,15
2,3,3,1,148,5.10,High/ Medium,Lean (170-185),No,€5.8M,247361,...,41,67,57,51,41,12,13,7,7,13
3,3,1,1,183,6.2,Medium/ Medium,Normal (185+),No,€4.2M,230872,...,37,41,12,21,18,70,64,63,64,68
4,4,3,1,154,5.9,High/ Medium,Normal (170-185),No,€46.2M,252145,...,45,75,73,76,73,7,8,8,12,15
5,3,3,2,161,5.11,High/ High,Lean (170-185),Yes,€99.4M,235212,...,46,80,71,75,76,10,8,14,6,8
6,3,2,1,165,5.11,High/ High,Normal (170-185),Yes,€15M,237000,...,33,59,65,71,68,6,14,10,6,11
7,2,5,1,174,6.2,High/ Medium,Lean (185+),No,€25M,245940,...,81,69,53,50,39,8,13,13,9,11
8,2,3,1,168,5.11,Medium/ High,Normal (170-185),No,€10.2M,222227,...,52,74,61,66,62,14,11,16,10,8
9,5,4,1,170,6.4,High/ Low,Unique,Yes,€68.5M,233731,...,75,77,25,21,18,7,6,9,7,11


In [31]:
full_data = pd.merge(data, detailed_data, how = 'inner', on = 'ID')

In [32]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [33]:
full_data.head(14)

,ID,picture,Nationality,name,age,position,overall,potential,club,value,wage,weak_foot,skill_moves,reputation,weight,height,Work Rate,Body Type,Real Face,release_clause,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Acceleration,Sprint Speed,Agility,Reactions,Balance,Shot Power,Jumping,Stamina,Strength,Long Shots,Aggression,Interceptions,Positioning,Vision,Penalties,composure,Defensive Awareness,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,255043,https://cdn.sofifa.com/players/255/043/21_60.png,United States,Daryl Dike,20,ST,69,81,Barnsley,€3.4M,€4K,4,2,1,220,6.2,High/ Low,Stocky (185+),No,NaN,30,69,64,57,47,64,33,33,35,66,66,74,53,67,50,80,71,59,96,60,60,15,66,50,65,52,15,15,10,11,9,15,11,13
1,228093,https://cdn.sofifa.com/players/228/093/21_60.png,France,Marcus Thuram,22,LM,79,85,Borussia Mönchengladbach,€28M,€37K,3,3,1,192,6.4,Medium/ Medium,Normal (185+),No,€53.2M,74,81,80,75,70,84,66,30,62,79,79,85,74,79,54,82,68,79,86,75,53,45,80,76,70,74,43,49,37,11,13,14,9,15
2,247361,https://cdn.sofifa.com/players/247/361/21_60.png,United States,Brenden Aaronson,19,CAM,70,81,FC Red Bull Salzburg,€3.4M,€9K,3,3,1,148,5.10,High/ Medium,Lean (170-185),No,€5.8M,55,64,37,70,51,72,50,35,64,72,80,76,82,63,80,61,53,71,50,63,53,51,68,69,41,67,57,51,41,12,13,7,7,13
3,230872,https://cdn.sofifa.com/players/230/872/21_60.png,Belgium,Mile Svilar,20,GK,67,78,SL Benfica,€1.9M,€2K,3,1,1,183,6.2,Medium/ Medium,Normal (185+),No,€4.2M,18,14,22,37,16,19,23,22,32,22,60,55,66,67,54,47,71,32,61,18,28,13,9,48,37,41,12,21,18,70,64,63,64,68
4,252145,https://cdn.sofifa.com/players/252/145/21_60.png,Portugal,Nuno Alexandre Tavares Mendes,18,LWB,77,87,Sporting CP,€21M,€4K,4,3,1,154,5.9,High/ Medium,Normal (170-185),No,€46.2M,74,45,67,71,54,72,66,59,68,75,82,86,82,76,76,80,65,76,69,63,67,75,72,59,45,75,73,76,73,7,8,8,12,15
5,235212,https://cdn.sofifa.com/players/235/212/21_60.png,Morocco,Achraf Hakimi,21,RM,83,88,Inter,€56M,€90K,3,3,2,161,5.11,High/ High,Lean (170-185),Yes,€99.4M,82,73,63,81,60,84,68,55,73,83,93,95,77,82,79,77,75,88,73,70,75,78,78,74,46,80,71,75,76,10,8,14,6,8
6,237000,https://cdn.sofifa.com/players/237/000/21_60.png,United States,Reggie Cannon,22,RWB,73,82,Boavista FC,€6.5M,€7K,3,2,1,165,5.11,High/ High,Normal (170-185),Yes,€15M,65,30,56,66,33,70,58,21,60,68,89,91,82,61,75,48,80,87,71,31,64,63,62,56,33,59,65,71,68,6,14,10,6,11
7,245940,https://cdn.sofifa.com/players/245/940/21_60.png,DR Congo,Silas Wamangituka,20,RM,75,85,VfB Stuttgart,€12.5M,€17K,2,5,1,174,6.2,High/ Medium,Lean (185+),No,€25M,66,79,65,70,65,83,59,33,48,77,85,92,84,70,73,70,60,72,73,49,62,54,77,58,81,69,53,50,39,8,13,13,9,11
8,222227,https://cdn.sofifa.com/players/222/227/21_60.png,Norway,Patrick Berg,22,CM,73,81,FK Bodø/Glimt,€7M,€5K,2,3,1,168,5.11,Medium/ High,Normal (170-185),No,€10.2M,64,59,45,77,52,70,66,63,74,73,76,77,72,72,78,67,58,90,70,63,63,69,67,74,52,74,61,66,62,14,11,16,10,8
9,233731,https://cdn.sofifa.com/players/233/731/21_60.png,Sweden,Alexander Isak,20,ST,80,85,Real Sociedad,€31.5M,€29K,5,4,1,170,6.4,High/ Low,Unique,Yes,€68.5M,61,83,75,77,81,78,70,57,47,81,82,83,79,78,68,84,71,70,75,76,52,29,84,76,75,77,25,21,18,7,6,9,7,11


In [34]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15075 entries, 0 to 15074
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   15075 non-null  object
 1   picture              14155 non-null  object
 2   Nationality          15075 non-null  object
 3   name                 15075 non-null  object
 4   age                  15075 non-null  object
 5   position             15075 non-null  object
 6   overall              15075 non-null  object
 7   potential            15075 non-null  object
 8   club                 14783 non-null  object
 9   value                15075 non-null  object
 10  wage                 15075 non-null  object
 11  weak_foot            15075 non-null  object
 12  skill_moves          15075 non-null  object
 13  reputation           15075 non-null  object
 14  weight               15075 non-null  object
 15  height               15075 non-null  object
 16  Work

In [35]:
full_data.to_csv("./fifa_new.csv")  